## Accessing Data from the MAAP

In this example, we demonstrate how to access data from the MAAP using the `getLocalPath()` function. At this time, this procedure is the same for user-contributed data added to the store.

We import the `os` module, import the `MAAP` package, and create a new MAAP class.

In [1]:
# import os module
import os

# import the MAAP package
from maap.maap import MAAP

# create MAAP class
maap = MAAP()

For this example, the `site_name` additional attribute is used to search for granules that have been tagged as part of the *Mondah Forest Gabon* research site. For more information about searching for granules in MAAP, please see https://maap-project.readthedocs.io/en/latest/search/granules.html.

In [2]:
# assign Mondah Forest Gabon site name
SITENAME = 'Mondah Forest Gabon'

# search for granules with site name
results = maap.searchGranule(site_name=SITENAME)

We assign a variable (in this case, `scene`) to the first result of our search from the cell above.

In [3]:
# grab first result
scene = results[0]

A data directory is then set, and if the directory does not already exist, it is created. The scene from our search is then downloaded into the file system in this directory. Here, the function `getLocalPath()` is accessing the data stored on the MAAP's Simple Storage Service (S3) bucket and downloading it directly to the path provided.

In [4]:
# set data directory
dataDir = './data'

# check if directory exists -> if directory doesn't exist, directory is created
if not os.path.exists(dataDir):
    os.mkdir(dataDir)

# download scene from search into data directory
data = scene.getLocalPath(dataDir)
data

'./data/Mondah_AGB_50m.tif'

Now we can see that the data directory has been created and the 'Mondah_AGB_50m.tif' scene is downloaded into the directory. The downloaded file remains in the data directory until the user deletes it.